In [ ]:
import pandas as pd
import numpy as np
import time
import yfinance as yf
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
stockData= yf.download('EURUSD=x',start='2023-09-01', interval='1h')

In [ ]:
stockData.reset_index(inplace=True) # moves Datatime index to column and creates new index column


In [ ]:
df=stockData
df.isna().sum() # check if any na values 
df=df[:10000]
#df.head(10)
 


In [ ]:
#cci = current candle  , acc = after cc, bcc= before cc, tdf= temp data frame
# we are only interested in values 1 & 2
def isPivot(cc,bcc,acc,tdf): 
    if (cc-bcc) < 0 or cc+acc >= len(tdf): 
        return 0 
    
    pivot_low=1
    pivot_high=1
 
    for x in range(cc-bcc, cc+acc+1):
        
        # checks if candels low and high are with in low and high of candle we comparing with and did not break them. 
        #We can ignore these ones so setting pivot_low = 0 and pivot_high = 0 accordingly 
        
        if (tdf.Low[cc]>tdf.Low[x]): 
            pivot_low=0
        if  (tdf.High[cc]<tdf.High[x]):
            pivot_high=0
            
    if pivot_low and pivot_high:
        return 3  #  if it makes higher wicker and lower wick, we can avoid this case. 
    elif pivot_low:
        return 1 
    elif pivot_high:
        return 2 
    else:
        return 0 # ignore this 
            
        
Pivoted_list=[]
candles_To_comare=10

for x in df.index:
    Pivoted_list.append(isPivot(x,candles_To_comare,candles_To_comare,df))
df['Pivoted']=Pivoted_list 

   


In [ ]:

#df.head(50)

In [ ]:
def signal_position(x):
    if x['Pivoted']==1:
        return x['Low']-0.001
    if x['Pivoted']==2:
        return x['High']+0.001
    else:
        return np.nan
    

signal_position_list=[]
for index, row in df.iterrows():
    signal_position_list.append(signal_position(row))
df['signal_pos'] = signal_position_list 


In [ ]:
df.head(50)

In [ ]:
# plot candle stick chart 



tdf = df[-500:-1]
fig = go.Figure(data=[go.Candlestick(x=tdf.index,
                open=tdf['Open'],
                high=tdf['High'],
                low=tdf['Low'],
                close=tdf['Close'],
                increasing_line_color= 'green', 
                decreasing_line_color= 'darkred')])


fig.add_scatter(x=tdf.index, y=tdf['signal_pos'], mode="markers",
                marker=dict(size=5, color="orange"),
                name="pivot")

fig.update_layout(
    xaxis_rangeslider_visible=False,
    yaxis_fixedrange=False,
    title='Plot Pivot Points | EURUSD | 1h'
)

fig.update_layout(paper_bgcolor='white', plot_bgcolor='black',    
                  xaxis=dict(showspikes=True),   
                  yaxis=dict(showspikes=True))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)


fig.show()

In [ ]:
import matplotlib.pyplot as plt

dfkeys = df[-1000:-1]

# Filter the dataframe based on the pivot column
high_values = dfkeys[dfkeys['Pivoted'] == 2]['High']
low_values = dfkeys[dfkeys['Pivoted'] == 1]['Low']

# Define the bin width
# Change this value as needed
bin_width = 0.002  # Change this value as needed

# Calculate the number of bins
bins = int((high_values.max() - low_values.min()) / bin_width)

# Create the histograms
plt.figure(figsize=(15, 7))
plt.hist(high_values, bins=bins, alpha=0.5,  color='red', label='Resistance')
plt.hist(low_values, bins=bins, alpha=0.5, color='blue', label='Support ')

plt.xlabel('Price')
plt.ylabel('No of Times Support & reisstance touched ')
plt.title('Histogram of support & resistance | EURUSD | 1h')
plt.legend(loc=2, fontsize=15)